lyft uber yellow year month day hour pulocationid weekday isholiday

In [1]:
import pandas as pd
# 讀取檔案資訊
datasite = '../TS6-201902_202306.parquet'
df = pd.read_parquet(datasite)

print(df.shape)
df.head()

(30273912, 13)


,Name,year,month,day,hour,PULocationID,weekday,is_holiday,count,lat,lon,TempTime,countN
0,lyft,2019,2,1,0,3,4,False,4,40.864294,-73.84651,2019-02-01 00:00:00,4.0
1,lyft,2019,2,1,1,3,4,False,2,40.864294,-73.84651,2019-02-01 01:00:00,2.0
2,lyft,2019,2,1,2,3,4,False,6,40.864294,-73.84651,2019-02-01 02:00:00,6.0
3,lyft,2019,2,1,3,3,4,False,1,40.864294,-73.84651,2019-02-01 03:00:00,1.0
4,lyft,2019,2,1,4,3,4,False,1,40.864294,-73.84651,2019-02-01 04:00:00,1.0


In [2]:
#清出基本格式
df_temp = df.copy()
df_temp = df_temp.drop(['Name','count','lat','lon','countN','hour','TempTime'],axis=1)
df_temp['TempTime'] = pd.to_datetime(df_temp[['year', 'month', 'day']])
df_temp.drop_duplicates(inplace=True)
df_temp = df_temp.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
print(df_temp.shape)
df_temp

(420471, 7)


,year,month,day,PULocationID,weekday,is_holiday,TempTime
0,2019,2,1,1,4,False,2019-02-01
1,2019,2,2,1,5,False,2019-02-02
2,2019,2,3,1,6,False,2019-02-03
3,2019,2,4,1,0,False,2019-02-04
4,2019,2,5,1,1,False,2019-02-05
...,...,...,...,...,...,...,...
420466,2023,6,26,263,0,False,2023-06-26
420467,2023,6,27,263,1,False,2023-06-27
420468,2023,6,28,263,2,False,2023-06-28
420469,2023,6,29,263,3,False,2023-06-29


In [3]:
#lyft
df_lyft = df[df['Name'] == 'lyft']
df_lyft = df_lyft.groupby(['Name', 'year', 'month', 'day', 'PULocationID'])['count'].sum().reset_index()
df_lyft['TempTime'] = pd.to_datetime(df_lyft[['year', 'month', 'day']])
df_lyft = df_lyft.drop(['Name','year','month','day'],axis=1)
df_lyft = df_lyft.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
df_lyft = df_lyft.drop(['PULocationID','TempTime'],axis=1)
df_lyft.rename(columns={'count': 'lyft'}, inplace=True)
#uber
df_uber = df[df['Name'] == 'uber']
df_uber = df_uber.groupby(['Name', 'year', 'month', 'day', 'PULocationID'])['count'].sum().reset_index()
df_uber['TempTime'] = pd.to_datetime(df_uber[['year', 'month', 'day']])
df_uber = df_uber.drop(['Name','year','month','day'],axis=1)
df_uber = df_uber.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
df_uber = df_uber.drop(['PULocationID','TempTime'],axis=1)
df_uber.rename(columns={'count': 'uber'}, inplace=True)
#yellow
df_yellow = df[df['Name'] == 'yellow']
df_yellow = df_yellow.groupby(['Name', 'year', 'month', 'day', 'PULocationID'])['count'].sum().reset_index()
df_yellow['TempTime'] = pd.to_datetime(df_yellow[['year', 'month', 'day']])
df_yellow = df_yellow.drop(['Name','year','month','day'],axis=1)
df_yellow = df_yellow.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
df_yellow = df_yellow.drop(['PULocationID','TempTime'],axis=1)
df_yellow.rename(columns={'count': 'yellow'}, inplace=True)

In [4]:
df_temp = pd.concat([df_lyft,df_uber,df_yellow, df_temp], axis=1)
df_temp = df_temp.drop(['TempTime'],axis=1)

In [5]:
df_temp.head()

,lyft,uber,yellow,year,month,day,PULocationID,weekday,is_holiday
0,0,3,11,2019,2,1,1,4,False
1,0,1,5,2019,2,2,1,5,False
2,0,0,9,2019,2,3,1,6,False
3,0,0,5,2019,2,4,1,0,False
4,2,0,6,2019,2,5,1,1,False


In [6]:
#放入地區
datasite_temp = '../taxi_zone_lat_lon/organized_taxi_zone_lat_lon.csv'
#創立一個 df_temp 讀經緯度資料
df_temp1 = pd.read_csv(datasite_temp)

df_temp1

,zone,LocationID,borough,lat,lon
0,Newark Airport,1,EWR,40.690243,-74.174270
1,Jamaica Bay,2,Queens,40.612163,-73.817643
2,Allerton/Pelham Gardens,3,Bronx,40.864294,-73.846510
3,Alphabet City,4,Manhattan,40.723853,-73.975209
4,Arden Heights,5,Staten Island,40.556678,-74.189803
...,...,...,...,...,...
258,Williamsburg (South Side),256,Brooklyn,40.710736,-73.962940
259,Woodlawn/Wakefield,259,Bronx,40.900107,-73.853635
260,Woodside,260,Queens,40.746439,-73.905907
261,World Trade Center,261,Manhattan,40.707456,-74.013983


In [7]:
df_temp = df_temp.merge(df_temp1[['LocationID', 'zone','borough']], left_on=['PULocationID'], right_on=['LocationID'], how='left')
df_temp = df_temp.drop(['LocationID'],axis = 1)
df_temp['datetime'] = pd.to_datetime(df_temp[['year', 'month', 'day']])
df_temp['total'] = df_temp['lyft'] + df_temp['uber'] + df_temp['yellow']
df_temp

,lyft,uber,yellow,year,month,day,PULocationID,weekday,is_holiday,zone,borough,datetime,total
0,0,3,11,2019,2,1,1,4,False,Newark Airport,EWR,2019-02-01,14
1,0,1,5,2019,2,2,1,5,False,Newark Airport,EWR,2019-02-02,6
2,0,0,9,2019,2,3,1,6,False,Newark Airport,EWR,2019-02-03,9
3,0,0,5,2019,2,4,1,0,False,Newark Airport,EWR,2019-02-04,5
4,2,0,6,2019,2,5,1,1,False,Newark Airport,EWR,2019-02-05,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
420466,880,2084,1621,2023,6,26,263,0,False,Yorkville West,Manhattan,2023-06-26,4585
420467,889,2205,1873,2023,6,27,263,1,False,Yorkville West,Manhattan,2023-06-27,4967
420468,910,2212,1829,2023,6,28,263,2,False,Yorkville West,Manhattan,2023-06-28,4951
420469,1064,2269,1857,2023,6,29,263,3,False,Yorkville West,Manhattan,2023-06-29,5190


In [8]:
df = df_temp
import os
# 是否輸出檔案 >0 輸出
df_save = 1
# 檔案輸出位置 
custom_output_path = f'.'
DataName = 'TS21-201902_202306'
# 輸出檔案
if df_save > 0 :
    os.makedirs(os.path.dirname(f'{custom_output_path}/{DataName}.parquet'), exist_ok=True)
    df.to_parquet(f'{custom_output_path}/{DataName}.parquet')
    print('save!')

save!


---